In [11]:
!ls

Client_Project_Ixchel.ipynb
Client_Project_John.ipynb
Client_Project_Kevin.ipynb
Ixchel_Twitter.ipynb
TwitterScraper_duplicates
Untitled.pdf
converting-text-files-to-combined-dictionary.ipynb
data
mycsv.csv
ny_temperature_Jan1.2017_Oct28.2017.ipynb
saved_tweets.csv
tweet
tweets_scrape_first_df.csv
twitter_credentials.json


In [10]:
import pandas as pd

In [25]:
df = pd.read_csv('tweets_scrape_first_df.csv')

In [26]:
df.drop(columns='Unnamed: 0', inplace=True)

In [27]:
df.head()

,ID,datetime,has_media,is_reply,is_retweet,medias,nbr_favorite,nbr_reply,nbr_retweet,text,url,user_id,usernameTweet
0,267716394043465728,2012-11-11 14:52:16,NaN,False,False,NaN,0.0,0.0,0.0,ConEd : NY Sandy power outages slip; costs...,/RealJezzy/status/267716394043465728,504279674.0,RealJezzy
1,298660085255770112,2013-02-05 00:11:27,NaN,False,False,NaN,0.0,0.0,0.0,@ ConEdison \n Power outage in queens,/meirBGNY/status/298660085255770112,909594764.0,meirBGNY
2,21860028236,2010-08-22 17:33:44,NaN,False,False,NaN,0.0,0.0,0.0,Power outage in the Whitestone section of Q...,/olgushka1/status/21860028236,68886571.0,olgushka1
3,1069021302667833344,2018-12-01 19:11:54,NaN,False,False,NaN,0.0,1.0,0.0,about that power outage that is now over.....,/jimcasale/status/1069021302667833344,12393522.0,jimcasale
4,767077182124486656,2016-08-20 15:13:46,NaN,False,False,NaN,3.0,0.0,7.0,.@ConEdison is responding to a power outag...,/NotifyNYC/status/767077182124486656,16145875.0,NotifyNYC


In [28]:
df.dtypes

ID                object
datetime          object
has_media         object
is_reply          object
is_retweet        object
medias            object
nbr_favorite     float64
nbr_reply        float64
nbr_retweet      float64
text              object
url               object
user_id          float64
usernameTweet     object
dtype: object

In [29]:
pd.to_datetime(df['datetime'][:12])

0    2012-11-11 14:52:16
1    2013-02-05 00:11:27
2    2010-08-22 17:33:44
3    2018-12-01 19:11:54
4    2016-08-20 15:13:46
5    2012-10-28 12:22:46
6    2018-03-02 22:08:23
7    2016-08-13 16:46:34
8    2016-08-13 16:38:28
9    2012-10-29 18:16:23
10   2012-10-31 04:19:06
11   2012-10-30 17:30:20
Name: datetime, dtype: datetime64[ns]

In [50]:
for i in df.datetime:
    if i[5:7] not in ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']:
        print(i)

20567184


In [52]:
df[df.datetime == '20567184']

,ID,datetime,has_media,is_reply,is_retweet,medias,nbr_favorite,nbr_reply,nbr_retweet,text,url,user_id,usernameTweet
2808,/Maggie4Obama/status/108170030474203136,20567184,Maggie4Obama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
df.iloc[2808]

ID               /Maggie4Obama/status/108170030474203136
datetime                                        20567184
has_media                                   Maggie4Obama
is_reply                                             NaN
is_retweet                                           NaN
medias                                               NaN
nbr_favorite                                         NaN
nbr_reply                                            NaN
nbr_retweet                                          NaN
text                                                 NaN
url                                                  NaN
user_id                                              NaN
usernameTweet                                        NaN
Name: 2808, dtype: object

In [56]:
df.drop(index=2808, inplace=True)

In [57]:
df[df.datetime == '20567184']

,ID,datetime,has_media,is_reply,is_retweet,medias,nbr_favorite,nbr_reply,nbr_retweet,text,url,user_id,usernameTweet


In [61]:
df.datetime = pd.to_datetime(df.datetime)

In [62]:
df.dtypes

ID                       object
datetime         datetime64[ns]
has_media                object
is_reply                 object
is_retweet               object
medias                   object
nbr_favorite            float64
nbr_reply               float64
nbr_retweet             float64
text                     object
url                      object
user_id                 float64
usernameTweet            object
dtype: object

In [64]:
df.isnull().sum()

ID                  0
datetime            0
has_media        4225
is_reply            0
is_retweet          0
medias           4225
nbr_favorite        0
nbr_reply           0
nbr_retweet         0
text                0
url                 1
user_id             1
usernameTweet       1
dtype: int64

#### Unique Values

In [93]:
unique = []
for i in df.columns:
    uniques = {}
    uniques['column'] = i 
    uniques['unique values'] = df[i].nunique()
    unique.append(uniques)
pd.DataFrame(unique)

,column,unique values
0,ID,4374
1,datetime,4254
2,has_media,1
3,is_reply,2
4,is_retweet,1
5,medias,49
6,nbr_favorite,33
7,nbr_reply,19
8,nbr_retweet,45
9,text,4116


# Pre-Processing

### Tokenize and Stem all of the Titles

In [94]:
# import tokenizer and stemmer
from nltk import word_tokenize
from nltk.stem import PorterStemmer

In [95]:
# define a function that takes in a title
# and tokenizes and stems that title
def stem_title(tweet):    
    stm = PorterStemmer()
    tokenized = word_tokenize(tweet)
    stem_tokes = []
    for toke in tokenized:
        stem_tokes.append(stm.stem(toke))
    stem_tokes

    comb = ''
    for stemmed in stem_tokes:
        comb += stemmed + ' '
    return comb

In [96]:
# check that the function works
stem_title(df.text[3])

"about that power outag that is now over ... from @ verizon `` the commerci power issu in your area may have been clear . If you still have issu with your verizon servic , click here . '' from @ conedison < cricket > "

In [97]:
# create a new column by applying the function to the title column
# and check out the head to see if it worked
df['stemmed_text'] = df.text.apply(stem_title)
df.head()

,ID,datetime,has_media,is_reply,is_retweet,medias,nbr_favorite,nbr_reply,nbr_retweet,text,url,user_id,usernameTweet,stemmed_text
0,267716394043465728,2012-11-11 14:52:16,NaN,False,False,NaN,0.0,0.0,0.0,ConEd : NY Sandy power outages slip; costs...,/RealJezzy/status/267716394043465728,504279674.0,RealJezzy,cone : NY sandi power outag slip ; cost could ...
1,298660085255770112,2013-02-05 00:11:27,NaN,False,False,NaN,0.0,0.0,0.0,@ ConEdison \n Power outage in queens,/meirBGNY/status/298660085255770112,909594764.0,meirBGNY,@ conedison power outag in queen
2,21860028236,2010-08-22 17:33:44,NaN,False,False,NaN,0.0,0.0,0.0,Power outage in the Whitestone section of Q...,/olgushka1/status/21860028236,68886571.0,olgushka1,power outag in the whiteston section of queen ...
3,1069021302667833344,2018-12-01 19:11:54,NaN,False,False,NaN,0.0,1.0,0.0,about that power outage that is now over.....,/jimcasale/status/1069021302667833344,12393522.0,jimcasale,about that power outag that is now over ... fr...
4,767077182124486656,2016-08-20 15:13:46,NaN,False,False,NaN,3.0,0.0,7.0,.@ConEdison is responding to a power outag...,/NotifyNYC/status/767077182124486656,16145875.0,NotifyNYC,. @ conedison is respond to a power outag in B...


In [98]:
# check out the tail, too
df.tail()

,ID,datetime,has_media,is_reply,is_retweet,medias,nbr_favorite,nbr_reply,nbr_retweet,text,url,user_id,usernameTweet,stemmed_text
4370,204532926447226880,2012-05-21 07:23:44,NaN,False,False,NaN,1.0,0.0,9.0,ConEd is responding to a power outage in ...,/NotifyNYC/status/204532926447226880,1.614588e+07,NotifyNYC,cone is respond to a power outag in SI zip cod...
4371,264374015714553858,2012-11-02 10:30:51,NaN,True,False,NaN,0.0,0.0,1.0,@ hhertzof Please check out our outage map ...,/ConEdison/status/264374015714553858,2.026208e+07,ConEdison,@ hhertzof pleas check out our outag map at ht...
4372,308557748671479808,2013-03-04 07:41:14,NaN,False,False,NaN,0.0,1.0,0.0,@ ConEdison Partial power outage in Green...,/curlycakes/status/308557748671479808,2.760942e+08,curlycakes,@ conedison partial power outag in greenwood b...
4373,969704423621648384,2018-03-02 17:42:05,NaN,False,False,NaN,0.0,1.0,0.0,"Just out of curiosity @EversourceCT , will the...",/yelenadasher/status/969704423621648384,2.791989e+09,yelenadasher,"just out of curios @ eversourcect , will there..."
4374,951772242203070464,2018-01-12 06:06:00,NaN,False,False,NaN,1.0,0.0,4.0,.@ConEdison is responding to a power outag...,/NotifyNYC/status/951772242203070464,1.614588e+07,NotifyNYC,. @ conedison is respond to a power outag in q...


# Model Prep

### CountVectorize

In [100]:
from sklearn.feature_extraction.text import CountVectorizer

In [111]:
# Instantiate
cv = CountVectorizer(stop_words='english', 
                     ngram_range=(1,1), 
                     max_features=1000
                    )

In [115]:
# Fit and Transform
sparse_df = cv.fit_transform(df.stemmed_text)

In [120]:
pd.DataFrame(sparse_df.todense())

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [113]:
cv.get_feature_names()

['000',
 '0075',
 '03',
 '032',
 '10',
 '100',
 '101',
 '10301',
 '10302',
 '10304',
 '10305',
 '10306',
 '10307',
 '10308',
 '10309',
 '10310',
 '10312',
 '10314',
 '10460',
 '10461',
 '10462',
 '10463',
 '10465',
 '10466',
 '10467',
 '10469',
 '10470',
 '10471',
 '11',
 '11004',
 '11203',
 '11204',
 '11205',
 '11206',
 '11207',
 '11208',
 '11209',
 '11210',
 '11214',
 '11219',
 '11223',
 '11224',
 '11228',
 '11229',
 '11234',
 '11235',
 '11236',
 '1131',
 '11355',
 '11356',
 '11357',
 '11358',
 '11364',
 '11365',
 '11367',
 '11368',
 '11373',
 '11377',
 '11378',
 '11379',
 '114',
 '11411',
 '11412',
 '11413',
 '11414',
 '11417',
 '11420',
 '11422',
 '11423',
 '11426',
 '11427',
 '11428',
 '11429',
 '11433',
 '11pm',
 '12',
 '13',
 '14th',
 '15',
 '16',
 '167',
 '17',
 '179',
 '1800',
 '1st',
 '1u2z88w',
 '20',
 '200',
 '2012',
 '2018',
 '22',
 '22lyehc',
 '24',
 '25',
 '250',
 '2500',
 '26633',
 '27',
 '29',
 '2h4jddc',
 '2nd',
 '30',
 '311',
 '34th',
 '36',
 '369',
 '39th',
 '40',
 

# Clustering

In [114]:
from sklearn.cluster import KMeans
km = KMeans()

In [121]:
km.fit(sparse_df)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=8, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)